In [1]:
# setup
%pip install litellm
%pip install groq


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
  Obtaining dependency information for groq from https://files.pythonhosted.org/packages/8a/13/8489c3df2047abed5acb4d3b188eedda7dc42d6c1fa4d853c22192de1115/groq-0.12.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.9/108.9 kB 5.2 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Insulin pump personal profile data
personal_profile_table = """
| Time Range   | Basal Rate (U/hr) | Correction Factor (1: mmol/L) | Carb Ratio (1: grams) | Target BG (mmol/L) |
|--------------|-------------------|------------------------------|-----------------------|---------------------|
| 12:00 AM     | 0.4               | 1:30                         | 1:10                  | 6.5                 |
| 2:00 AM      | 0.5               | 1:30                         | 1:10                  | 6.5                 |
| 5:00 AM      | 0.8               | 1:30                         | 1:10                  | 6.5                 |
| 6:00 AM      | 0.9               | 1:30                         | 1:10                  | 5.6                 |
| 7:00 AM      | 0.8               | 1:30                         | 1:10                  | 5.6                 |
| 8:00 AM      | 0.8               | 1:25                         | 1:10                  | 5.6                 |
| 9:00 AM      | 0.45              | 1:25                         | 1:10                  | 5.6                 |
| 12:00 PM     | 0.45              | 1:25                         | 1:10                  | 5.6                 |
| 1:00 PM      | 0.5               | 1:10                         | 1:10                  | 5.6                 |
| 3:00 PM      | 0.5               | 1:10                         | 1:10                  | 5.6                 |
| 4:00 PM      | 0.6               | 1:25                         | 1:10                  | 5.6                 |
| 7:00 PM      | 0.45              | 1:25                         | 1:12                  | 5.6                 |
| 9:00 PM      | 0.3               | 1:30                         | 1:10                  | 5.6                 |
| 11:00 PM     | 0.3               | 1:30                         | 1:10                  | 5.6                 |
"""

personal_profile_json = """
{
  "timed_settings": [
    {"time_range": "12:00 AM", "basal_rate": 0.4, "correction_factor": "1:30", "carb_ratio": "1:10", "target_bg": 6.5},
    {"time_range": "2:00 AM", "basal_rate": 0.5, "correction_factor": "1:30", "carb_ratio": "1:10", "target_bg": 6.5},
    {"time_range": "5:00 AM", "basal_rate": 0.8, "correction_factor": "1:30", "carb_ratio": "1:10", "target_bg": 6.5},
    {"time_range": "6:00 AM", "basal_rate": 0.9, "correction_factor": "1:30", "carb_ratio": "1:10", "target_bg": 5.6},
    {"time_range": "7:00 AM", "basal_rate": 0.8, "correction_factor": "1:30", "carb_ratio": "1:10", "target_bg": 5.6},
    {"time_range": "8:00 AM", "basal_rate": 0.8, "correction_factor": "1:25", "carb_ratio": "1:10", "target_bg": 5.6},
    {"time_range": "9:00 AM", "basal_rate": 0.45, "correction_factor": "1:25", "carb_ratio": "1:10", "target_bg": 5.6},
    {"time_range": "12:00 PM", "basal_rate": 0.45, "correction_factor": "1:25", "carb_ratio": "1:10", "target_bg": 5.6},
    {"time_range": "1:00 PM", "basal_rate": 0.5, "correction_factor": "1:10", "carb_ratio": "1:10", "target_bg": 5.6},
    {"time_range": "3:00 PM", "basal_rate": 0.5, "correction_factor": "1:10", "carb_ratio": "1:10", "target_bg": 5.6},
    {"time_range": "4:00 PM", "basal_rate": 0.6, "correction_factor": "1:25", "carb_ratio": "1:10", "target_bg": 5.6},
    {"time_range": "7:00 PM", "basal_rate": 0.45, "correction_factor": "1:25", "carb_ratio": "1:12", "target_bg": 5.6},
    {"time_range": "9:00 PM", "basal_rate": 0.3, "correction_factor": "1:30", "carb_ratio": "1:10", "target_bg": 5.6},
    {"time_range": "11:00 PM", "basal_rate": 0.3, "correction_factor": "1:30", "carb_ratio": "1:10", "target_bg": 5.6}
  ]
}
"""

personal_profile_xml = """
<insulin_pump_timed_settings>
  <setting>
    <time_range>12:00 AM</time_range>
    <basal_rate>0.4</basal_rate>
    <correction_factor>1:30</correction_factor>
    <carb_ratio>1:10</carb_ratio>
    <target_bg>6.5</target_bg>
  </setting>
  <setting>
    <time_range>2:00 AM</time_range>
    <basal_rate>0.5</basal_rate>
    <correction_factor>1:30</correction_factor>
    <carb_ratio>1:10</carb_ratio>
    <target_bg>6.5</target_bg>
  </setting>
  <setting>
    <time_range>5:00 AM</time_range>
    <basal_rate>0.8</basal_rate>
    <correction_factor>1:30</correction_factor>
    <carb_ratio>1:10</carb_ratio>
    <target_bg>6.5</target_bg>
  </setting>
  <setting>
    <time_range>6:00 AM</time_range>
    <basal_rate>0.9</basal_rate>
    <correction_factor>1:30</correction_factor>
    <carb_ratio>1:10</carb_ratio>
    <target_bg>5.6</target_bg>
  </setting>
  <setting>
    <time_range>7:00 AM</time_range>
    <basal_rate>0.8</basal_rate>
    <correction_factor>1:30</correction_factor>
    <carb_ratio>1:10</carb_ratio>
    <target_bg>5.6</target_bg>
  </setting>
  <setting>
    <time_range>8:00 AM</time_range>
    <basal_rate>0.8</basal_rate>
    <correction_factor>1:25</correction_factor>
    <carb_ratio>1:10</carb_ratio>
    <target_bg>5.6</target_bg>
  </setting>
  <setting>
    <time_range>9:00 AM</time_range>
    <basal_rate>0.45</basal_rate>
    <correction_factor>1:25</correction_factor>
    <carb_ratio>1:10</carb_ratio>
    <target_bg>5.6</target_bg>
  </setting>
  <setting>
    <time_range>12:00 PM</time_range>
    <basal_rate>0.45</basal_rate>
    <correction_factor>1:25</correction_factor>
    <carb_ratio>1:10</carb_ratio>
    <target_bg>5.6</target_bg>
  </setting>
  <setting>
    <time_range>1:00 PM</time_range>
    <basal_rate>0.5</basal_rate>
    <correction_factor>1:10</correction_factor>
    <carb_ratio>1:10</carb_ratio>
    <target_bg>5.6</target_bg>
  </setting>
  <setting>
    <time_range>3:00 PM</time_range>
    <basal_rate>0.5</basal_rate>
    <correction_factor>1:10</correction_factor>
    <carb_ratio>1:10</carb_ratio>
    <target_bg>5.6</target_bg>
  </setting>
  <setting>
    <time_range>4:00 PM</time_range>
    <basal_rate>0.6</basal_rate>
    <correction_factor>1:25</correction_factor>
    <carb_ratio>1:10</carb_ratio>
    <target_bg>5.6</target_bg>
  </setting>
  <setting>
    <time_range>7:00 PM</time_range>
    <basal_rate>0.45</basal_rate>
    <correction_factor>1:25</correction_factor>
    <carb_ratio>1:12</carb_ratio>
    <target_bg>5.6</target_bg>
  </setting>
  <setting>
    <time_range>9:00 PM</time_range>
    <basal_rate>0.3</basal_rate>
    <correction_factor>1:30</correction_factor>
    <carb_ratio>1:10</carb_ratio>
    <target_bg>5.6</target_bg>
  </setting>
  <setting>
    <time_range>11:00 PM</time_range>
    <basal_rate>0.3</basal_rate>
    <correction_factor>1:30</correction_factor>
    <carb_ratio>1:10</carb_ratio>
    <target_bg>5.6</target_bg>
  </setting>
</insulin_pump_timed_settings>
"""

In [ ]:
# Integration with groq
from litellm import completion

system_message = (
    "You are a medical assistant that analyzes continuous glucose monitoring (CGM) data to provide personalized recommendations for insulin pump adjustments."
    "Based on glucose trends, make data-driven suggestions to optimize insulin delivery rates, focusing on preventing both hyperglycemia and hypoglycemia."
)

alarms_data = ""
cgm_data = ""
bolus_data = ""
insulin_per_day_data = ""
basal_data = ""
user_message = f"""
"""
response = completion(
    model="groq/llama-3.1-8b-instant", 
    messages=[
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_message}
   ],
)
print(response.choices[0].message.content)